# Pokémon Battle Ontology Initialization

### Made by Colin Bouwmeester, Jonas Heller and Musa Karim

#### Make sure to have the provided "Pokemon.csv" file in the same directory as this .ipynb file.
#### Be sure to have installed all the imported modules, running 'pip install modulename'. The used modules are: 'SPARQLWrapper', 'pandas' and 'urllib.parse'




#### If you run every cell, a Turtle file named "Pokemon.ttl" should appear in your directory. Be mindfull of the fact that this file is not reasoned over yet and this still has to happen in, for instance, Protégé.





In [17]:
#module imports
from rdflib import Graph, RDF, Namespace, Literal, URIRef
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import urllib.parse    

In [18]:
#import csv, define types and other data
pokemon = pd.read_csv("Pokemon.csv")

normal = "Normal"
fighting = "Fighting"
flying = "Flying"
poison = "Poison"
ground = "Ground"
rock = "Rock"
bug = "Bug"
ghost = "Ghost"
steel = "Steel"
fire = "Fire"
water = "Water"
grass = "Grass"
electric = "Electric"
psychic = "Psychic"
ice = "Ice"
dragon = "Dragon"
dark = "Dark"
fairy = "Fairy"

totallist = [normal, fighting, flying, poison, ground, rock, bug, ghost, steel, fire, water, grass,
             electric, psychic, ice, dragon, dark, fairy]

weak_dict = {normal:[rock,steel], 
            fighting:[flying,poison,bug,psychic,fairy],
            flying:[rock,steel,electric],
            poison:[poison,ground,rock,ghost],
            ground:[bug,grass],
            rock:[fighting,ground,steel],
            bug:[fighting,flying,poison,ghost,steel,fire,fairy],
            ghost:[dark],
            steel:[steel,fire,water],
            fire:[rock,fire,water,dragon],
            water:[water,grass,dragon],
            grass:[flying,poison,bug,steel,fire,grass,dragon],
            electric:[grass,electric,dragon],
            psychic:[steel,psychic],
            ice:[steel,fire,water,ice],
            dragon:[steel],
            dark:[fighting,dark,fairy],
            fairy:[poison,steel,fire]
            }


ineffective_dict = {normal:[ghost],
                   fighting:[ghost],
                   flying:[],
                   poison:[steel],
                   ground:[flying],
                   rock:[],
                   bug:[],
                   ghost:[normal],
                   steel:[],
                   fire:[],
                   water:[],
                   grass:[],
                   electric:[ground],
                   psychic:[dark],
                   ice:[],
                   dragon:[fairy],
                   dark:[],
                   fairy:[]}


strong_dict = {normal:[],
                 fighting:[normal,rock,steel,ice,dark],
                 flying:[fighting,bug,grass],
                 poison:[grass,fairy],
                 ground:[poison,rock,steel,fire,electric],
                 rock:[flying,bug,fire,ice],
                 bug:[grass,psychic,dark],
                 ghost:[ghost,psychic],
                 steel:[rock,ice,fairy],
                 fire:[bug,steel,grass,ice],
                 water:[ground,rock,fire],
                 grass:[ground,rock,water],
                 electric:[flying,water],
                 psychic:[fighting,poison],
                 ice:[flying,ground,grass,dragon],
                 dragon:[dragon],
                 dark:[ghost,psychic],
                 fairy:[fighting,dragon,dark]}

gen_list = ['Kanto', 'Johto', 'Hoenn', 'Sinnoh', 'Unova', 'Kalos']

trainer_dict = {'Ash': ['Pikachu', 'Charizard'], 'Brock' : ['Onix'],
                'Misty' : ['Togepi', 'Psyduck'], 'Gary' : ['Blastoise', 'Arcanine']}

pokeball_dict = {'Ash' : ['Pokéball', 'Greatball', 'Ultraball'], 'Brock' : ['Pokéball', 'Greatball'],
                 'Misty' : ['Pokéball'], 'Gary' : ['Greatball', 'Ultraball']}


neutral_dict = {typing: [x for x in totallist if x not in (weak_dict[typing] + strong_dict[typing]
                                                           + ineffective_dict[typing])] for typing in totallist}


In [19]:
#Initialize graph and namespaces
pokegraph = Graph()
DBR = Namespace('http://dbpedia.org/resource/')
DBO = Namespace('http://dbpedia.org/ontology/')
RDFS = Namespace('http://www.w3.org/2000/01/rdf-schema#')
DUL = Namespace('http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#')
POK = Namespace('http://www.example.org/pokemon/')
BIBO = Namespace('http://purl.org/ontology/bibo/')
OWL = Namespace("http://www.w3.org/2002/07/owl#")
FOAF = Namespace("http://xmlns.com/foaf/0.1/")
PEXT = Namespace("http://www.ontotext.com/proton/protonext#")

pokegraph.bind('foaf', FOAF)
pokegraph.bind('pok',POK)
pokegraph.bind('dbr',DBR)
pokegraph.bind('dbo',DBO)
pokegraph.bind('rdfs',RDFS)
pokegraph.bind('dul',DUL)
pokegraph.bind('bibo',BIBO)
pokegraph.bind('owl',OWL)
pokegraph.bind('pext',PEXT)

#Add classes
pokegraph.add((POK.Pokémon, RDF.type, RDFS.Class))
pokegraph.add((POK.Region, RDF.type, RDFS.Class))
pokegraph.add((POK.Typing, RDF.type, RDFS.Class))
pokegraph.add((POK.FantasyWorld, RDF.type, RDFS.Class))
pokegraph.add((POK.PokémonWorld, RDF.type, RDFS.Class))
pokegraph.add((POK.Creature, RDF.type, RDFS.Class))
pokegraph.add((POK.Trainer, RDF.type, RDFS.Class))
pokegraph.add((POK.Item, RDF.type, RDFS.Class))
pokegraph.add((POK.Pokéball, RDF.type, RDFS.Class))
pokegraph.add((POK.Image, RDF.type, RDFS.Class))
pokegraph.add((POK.Sound, RDF.type, RDFS.Class))
pokegraph.add((PEXT.Human, RDF.type, RDFS.Class))
pokegraph.add((PEXT.Mammal, RDF.type, RDFS.Class))
pokegraph.add((POK.RealWorld, RDF.type, RDFS.Class))
pokegraph.add((POK.Media, RDF.type, RDFS.Class))

#Setup class hierarchy

pokegraph.add((POK.Media, RDFS.subClassOf, POK.RealWorld))
pokegraph.add((POK.Sound, RDFS.subClassOf, POK.Media))
pokegraph.add((POK.Image, RDFS.subClassOf, POK.Media))
pokegraph.add((POK.PokémonWorld, RDFS.subClassOf, POK.FantasyWorld))
pokegraph.add((POK.Creature, RDFS.subClassOf, POK.PokémonWorld))
pokegraph.add((PEXT.Mammal, RDFS.subClassOf, POK.Creature))
pokegraph.add((PEXT.Human, RDFS.subClassOf, PEXT.Mammal))
pokegraph.add((POK.Trainer, RDFS.subClassOf, PEXT.Human))
pokegraph.add((POK.Pokémon, RDFS.subClassOf, POK.Creature))
pokegraph.add((POK.Item, RDFS.subClassOf, POK.PokémonWorld))
pokegraph.add((POK.Pokéball, RDFS.subClassOf, POK.Item))
pokegraph.add((POK.Region, RDFS.subClassOf, POK.PokémonWorld))
pokegraph.add((POK.Typing, RDFS.subClassOf, POK.PokémonWorld))

#Load Pokémon info from csv and other places
for i in range(len(list(pokemon['Name']))):
    subject = URIRef("http://www.example.org/pokemon/" + pokemon['Name'][i].replace(' ', '_').replace('.', '').replace('\'',''))

    #Load name, region and first type
    pokegraph.add((subject, FOAF.name, Literal(pokemon['Name'][i].replace(' ', '_').replace('.', '').replace('\'',''))))
    pokegraph.add((subject, POK.hasRegion, URIRef("http://www.example.org/pokemon/" + gen_list[int(pokemon['Generation'][i])-1])))
    pokegraph.add((subject, POK.hasType, URIRef("http://www.example.org/pokemon/" + pokemon['Type 1'][i])))
    pokegraph.add((subject, POK.stats, URIRef("http://www.example.org/pokemon/" + str(pokemon['Total'][i]))))
    
    
    #Load type 2 (optional since some Pokémon only have one type)
    if type(pokemon['Type 2'][i]) is not float:
        pokegraph.add((subject, POK.hasType, URIRef("http://www.example.org/pokemon/" + pokemon['Type 2'][i])))
        
        
    #Load Image
    ID = str(i+1)
    image = URIRef("https://pokeres.bastionbot.org/images/pokemon/" + ID + ".png")
    pokegraph.add((subject, BIBO.Image, image))
    
    #Load Sound
    if i < 9:
        encode = URIRef(urllib.parse.quote('sound/00' + str(i+1) + '-' + pokemon['Name'][i].replace(' ', '_').replace('.', '').replace('\'','') + '.wav'))
        pokegraph.add((subject, BIBO.AudioDocument, encode))
    elif i < 99:
        encode = URIRef(urllib.parse.quote('sound/0' + str(i+1) + '-' + pokemon['Name'][i].replace(' ', '_').replace('.', '').replace('\'','') + '.wav'))
        pokegraph.add((subject, BIBO.AudioDocument, encode))
    else:
        encode = URIRef(urllib.parse.quote('sound/' + str(i+1) + '-' + pokemon['Name'][i].replace(' ', '_').replace('.', '').replace('\'','') + '.wav'))
        pokegraph.add((subject, BIBO.AudioDocument, encode))
        
#Load Trainer Pokéballs and Pokémon
for trainer in trainer_dict:
    trainerURI = URIRef("http://www.example.org/pokemon/" + trainer)
    
    for balltype in pokeball_dict[trainer]:
        pokeball = URIRef('http://www.example.org/pokemon/' + balltype)
        pokegraph.add((trainerURI, POK.hasItem, pokeball))
    
    for pok in trainer_dict[trainer]:
        subject = URIRef("http://www.example.org/pokemon/" + pok.replace(' ', '_').replace('.', ''))      
        pokegraph.add((trainerURI, POK.hasPokemon, subject))
        
#Load type advantages
for typing in totallist:
    typingURI = URIRef("http://www.example.org/pokemon/" + typing)
    
    for strong in strong_dict[typing]:
        pokegraph.add((typingURI, POK.StrongTo, URIRef("http://www.example.org/pokemon/" + strong)))
    for weak in weak_dict[typing]:
        pokegraph.add((typingURI, POK.WeakTo, URIRef("http://www.example.org/pokemon/" + weak)))
    for resist in ineffective_dict[typing]:
        pokegraph.add((typingURI, POK.WeakTo, URIRef("http://www.example.org/pokemon/" + resist)))
    for neutral in neutral_dict[typing]:
        pokegraph.add((typingURI, POK.NeutralTo, URIRef("http://www.example.org/pokemon/" + neutral)))
        
#Define object properties    
pokegraph.add((POK.hasType, RDF.type, OWL.ObjectProperty))    
pokegraph.add((POK.hasRegion, RDF.type, OWL.ObjectProperty))
pokegraph.add((FOAF.name, RDF.type, OWL.ObjectProperty))  
pokegraph.add((BIBO.Image, RDF.type, OWL.ObjectProperty))
pokegraph.add((BIBO.AudioDocument, RDF.type, OWL.ObjectProperty))
pokegraph.add((POK.hasPokemon, RDF.type, OWL.ObjectProperty))
pokegraph.add((POK.hasItem, RDF.type, OWL.ObjectProperty))
pokegraph.add((POK.StrongTo, RDF.type, OWL.ObjectProperty))
pokegraph.add((POK.WeakTo, RDF.type, OWL.ObjectProperty))
pokegraph.add((POK.NeutralTo, RDF.type, OWL.ObjectProperty))
pokegraph.add((POK.stats, RDF.type, OWL.ObjectProperty))


#Define domains and ranges
pokegraph.add((POK.hasType, RDFS.domain, POK.Pokémon))
pokegraph.add((POK.hasType, RDFS.range, POK.Typing))
pokegraph.add((POK.hasRegion, RDFS.range, POK.Region))
pokegraph.add((BIBO.Image, RDFS.range, POK.Image))
pokegraph.add((BIBO.AudioDocument, RDFS.range, POK.Sound))
pokegraph.add((POK.hasPokemon, RDFS.domain, POK.Trainer))
pokegraph.add((POK.hasItem, RDFS.range, POK.Pokéball))

#Print output to verify    
print(pokegraph.serialize(format='turtle').decode("utf-8"))

#Write ontology tot turtlefile
pokegraph.serialize(destination="Pokemon.ttl", format='turtle')

@prefix bibo: <http://purl.org/ontology/bibo/> .
@prefix dbo: <http://dbpedia.org/ontology/> .
@prefix dbr: <http://dbpedia.org/resource/> .
@prefix dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix pext: <http://www.ontotext.com/proton/protonext#> .
@prefix pok: <http://www.example.org/pokemon/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

pok:Creature a rdfs:Class ;
    rdfs:subClassOf pok:PokémonWorld .

pok:FantasyWorld a rdfs:Class .

pok:Image a rdfs:Class ;
    rdfs:subClassOf pok:Media .

pok:Item a rdfs:Class ;
    rdfs:subClassOf pok:PokémonWorld .

pok:Media a rdfs:Class ;
    rdfs:subClassOf pok:RealWorld .

pok:Pokéball a rdfs:Class ;
    rdfs:subClassOf pok:Item .

pok:Pokémon a rdf